In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import random
random.seed(19401)

In [ ]:
def visualize_point_cloud(point_cloud):
    o3d.visualization.draw_geometries([point_cloud], window_name='point cloud')

def save_point_cloud(point_cloud, path):
    o3d.io.write_point_cloud(path, point_cloud, write_ascii=True)

In [ ]:
import numpy as np
import cv2
from scipy.spatial import cKDTree

def depth_to_point_cloud(depth_map, original_points, focal_length):
    height, width = depth_map.shape
    center_x = width / 2
    center_y = height / 2
    points = []

    for v in range(height):
        for u in range(width):
            Z = depth_map[v, u]
            if Z == 0:  # 跳過無效點
                continue
            X = (u - center_x) * Z / focal_length
            Y = (v - center_y) * Z / focal_length
            points.append([X, Y, Z])

    points = np.array(points)

    # 取得原始點雲的中心
    original_center = np.mean(original_points, axis=0)

    # 取得恢復點雲的中心
    recovered_center = np.mean(points, axis=0)

    # 調整恢復點雲使其與原始點雲對齊
    aligned_points = points + (original_center - recovered_center)

    return aligned_points # numpy

def find_corresponding_points(recovered_points, original_points):
    # 創建一個KD樹來加速最近鄰搜索
    tree = cKDTree(original_points)

    # 查找最近的點
    distances, indices = tree.query(recovered_points)
    
    # 使用最近的點的索引來獲取對應的點
    corresponding_points = original_points[indices]
    
    return corresponding_points

In [ ]:
def adjust_z_position(points, adjustment):
    adjusted_points = points.copy()
    adjusted_points[:, 2] += adjustment
    return adjusted_points

def gausian_z_position(points, mean, std):
    gausian_points = points.copy()
    mean = mean
    std_dev = std
    noise = np.random.normal(mean, std_dev, gausian_points.shape[0])
    gausian_points[:, 2] += noise
    return gausian_points
    
def expand_region(points, original_points, k=10):
    tree = cKDTree(original_points)
    _, indices = tree.query(points, k=k)
    expanded_points = original_points[indices.flatten()]
    return expanded_points

In [ ]:
import os
import glob
import gzip
from PIL import Image

depth_file_dirs = ["F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_depth\\mask"]
normal_file_dirs = ["F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_normal\\mask"]

# 設定要儲存影像的目標資料夾
# os.makedirs('F:\\PointCloud_Attack\\adv_data\\adv_depth', exist_ok=True)
# os.makedirs('F:\\PointCloud_Attack\\adv_data\\adv_normal', exist_ok=True)

clean_depth_path = 'E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Depth_Nu_only'
clean_normal_path = 'E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Normal_Nu_only'
point_cloud_path = 'E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\pointcloud'
focal_length = 500  # 假設焦距
adjustment_value = 100
mean = 0
std = 5

In [ ]:
# 使用 glob 列出 heatmap 資料夾  
for file_dir in depth_file_dirs:
    depth_files = glob.glob(os.path.join(file_dir, "*"))
    for iteration in depth_files:
        img_path = glob.glob(os.path.join(iteration, "*_nu*"))
        for depth_files_path in img_path:
            label = os.path.basename(depth_files_path).split('d')[0]
            name = os.path.basename(depth_files_path).split('_nu.png')[0]
            
            #  點雲移動
            original_point_cloud = o3d.io.read_point_cloud(os.path.join(point_cloud_path, label, name + '.xyz'), format='xyz')
            original_points = np.asarray(original_point_cloud.points)
            depth_map = cv2.imread(depth_files_path, cv2.IMREAD_UNCHANGED)
            
            # 計算恢復的點雲並與原始點雲對齊 
            recovered_points = depth_to_point_cloud(depth_map, original_points, focal_length)
            corresponding_points = find_corresponding_points(recovered_points, original_points)    
            
            modify_point = original_points.copy()
            expanded_points = expand_region(corresponding_points, modify_point, k=10)
            # 將點雲數據轉換為集合
            set_A = set(map(tuple, modify_point))
            set_B = set(map(tuple, expanded_points))
            set_diff = set_A - set_B
            
            # 將結果轉換回 NumPy 
            A_minus_B = np.array(list(set_diff))
            set_B = np.array(list(set_B))

            # adjust_z = adjust_z_position(set_B, adjustment_value)
            gausian_z = gausian_z_position(corresponding_points, mean, std)
            # new_points = np.vstack((A_minus_B, adjust_z))
            new_points = np.vstack((A_minus_B, gausian_z))
            
            label_path = os.path.join('F:\\PointCloud_Attack\\adv_data\\pc_adv_mean=0_std=5', label)
            os.makedirs(label_path, exist_ok=True)
            dst = os.path.join(label_path, name)
            
            point_cloud = o3d.geometry.PointCloud()
            point_cloud.points = o3d.utility.Vector3dVector(new_points)
            save_point_cloud(point_cloud, path = dst + '.xyz')